In [1]:
import requests
import pandas as pd
import json
import re
import datetime

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [2]:
#url = 'https://api.queryly.com/v4/search.aspx?queryly_key=62d9c40063044c14&initialized=1&&query=milei&endindex=0&batchsize=10&callback=&extendeddatafields=creator,imageresizer,promo_image&timezoneoffset'
#qry = requests.get(url).text
#json_str_match = re.search(r'JSON\.parse\(\'(.*?)\'\)', qry, re.DOTALL)
#print(json_str_match)

In [3]:
rangos = range(0,1050,30)
resultados = []

for endindex in rangos:
    url = f"https://api.queryly.com/v4/search.aspx?queryly_key=62d9c40063044c14&initialized={0}&&query=milei&endindex={endindex}&batchsize={30}&callback=&extendeddatafields=creator,imageresizer,promo_image&timezoneoffset"
    response = requests.get(url)
    content = response.text
    json_str_match = re.search(r'results \= JSON\.parse\(\'(.*?)\'\)', content, re.DOTALL)
    if json_str_match:
        json_str = json_str_match.group(1)
        json_str = json_str.replace('\\\\','\\')
        json_str = json_str.replace('\\\'','')
        resultados.append(json_str)
    else:
        data = "No JSON found in the script."
    

In [4]:
fallos = []
i = 0
for resultado in resultados:
    i += 1
    try:
        data = json.loads(resultado)
    except:
        fallos.append(i)

In [5]:
df = [pd.DataFrame(data['items']) for resultado in resultados]
dfs = pd.concat(df, ignore_index=True)


In [6]:
noticias = pd.DataFrame({
    'indice': dfs.index,
    'titulo': dfs.title,
    'fecha': pd.to_datetime(pd.to_numeric(dfs.pubdateunix), unit='s'),
    'link': dfs.link,
    'contenido': "",
    'copete': dfs.description
})

In [19]:
noticias.to_excel('03_Infobae.xlsx')
noticias['contenido'] = None

In [8]:
Options = Options()
Options.add_argument('--headless')
driver = webdriver.Chrome(options=Options)

In [9]:
url_base = 'https://www.infobae.com/'

In [32]:
soup = []
limite = 500
i = 0
for index, row in noticias.iterrows():
    try:
        if i > limite:
            continue
        if row.contenido is None:
            print(f"indice {index}| link: {url_base}{row.link}", end='\r')
            driver.get(f"{url_base}{row.link}")
            pag =  driver.page_source
            soup = BeautifulSoup(pag, 'html5lib')
            #noticias.at[index,'source'] = soup
            contenido_source = soup.find('div', class_='body-article')
            contenido = ''
            for parrafo in contenido_source.find_all('p'):
                contenido = contenido + parrafo.get_text() + '\n'
            noticias.at[index,'contenido'] = contenido
            i += 1
    except:
        noticias.at[index,'source'] = None

In [33]:
#noticias.drop('source', axis = 1, inplace=True)
noticias.to_excel('03_Infobae.xlsx')